In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import matplotlib.gridspec as gridspec

In [2]:
%%time
train_metadata_kaggle = dd.read_csv('klm_train.csv')
test_metadata_kaggle = dd.read_csv('klm_test.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 64) (3492890, 63)
CPU times: user 1min 46s, sys: 6.72 s, total: 1min 53s
Wall time: 20 s


In [3]:
#Always seed the randomness of this universe
np.random.seed(51)

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata = dd.read_csv('cesium_train.csv')
test_metadata = dd.read_csv('cesium_test.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 288) (3492890, 283)
CPU times: user 6min 22s, sys: 23.6 s, total: 6min 45s
Wall time: 1min 30s


In [5]:
test_metadata.columns = [x.replace("('","__").replace("',","___").replace(")","_").replace(' ','') for x in test_metadata.columns]

In [6]:
drop_from_train_metadata = [x for x in train_metadata.columns if x not in test_metadata.columns]

In [7]:
%%time
train_metadata.drop(drop_from_train_metadata,axis=1,inplace=True)

CPU times: user 3.36 ms, sys: 0 ns, total: 3.36 ms
Wall time: 4.24 ms


In [8]:
print(train_metadata.shape,test_metadata.shape)

(7848, 283) (3492890, 283)


In [11]:
train_metadata.head()

,object_id,__amplitude___0_,__amplitude___1_,__amplitude___2_,__amplitude___3_,__amplitude___4_,__amplitude___5_,__flux_percentile_ratio_mid20___0_,__flux_percentile_ratio_mid20___1_,__flux_percentile_ratio_mid20___2_,__flux_percentile_ratio_mid20___3_,__flux_percentile_ratio_mid20___4_,__flux_percentile_ratio_mid20___5_,__flux_percentile_ratio_mid35___0_,__flux_percentile_ratio_mid35___1_,__flux_percentile_ratio_mid35___2_,__flux_percentile_ratio_mid35___3_,__flux_percentile_ratio_mid35___4_,__flux_percentile_ratio_mid35___5_,__flux_percentile_ratio_mid50___0_,__flux_percentile_ratio_mid50___1_,__flux_percentile_ratio_mid50___2_,__flux_percentile_ratio_mid50___3_,__flux_percentile_ratio_mid50___4_,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___0_,__flux_percentile_ratio_mid65___1_,__flux_percentile_ratio_mid65___2_,__flux_percentile_ratio_mid65___3_,__flux_percentile_ratio_mid65___4_,__flux_percentile_ratio_mid65___5_,__flux_percentile_ratio_mid80___0_,__flux_percentile_ratio_mid80___1_,__flux_percentile_ratio_mid80___2_,__flux_percentile_ratio_mid80___3_,__flux_percentile_ratio_mid80___4_,__flux_percentile_ratio_mid80___5_,__max_slope___0_,__max_slope___1_,__max_slope___2_,__max_slope___3_,__max_slope___4_,__max_slope___5_,__linear_trend___0_,__linear_trend___1_,__linear_trend___2_,__linear_trend___3_,__linear_trend___4_,__linear_trend___5_,__median_absolute_deviation___0_,__median_absolute_deviation___1_,__median_absolute_deviation___2_,__median_absolute_deviation___3_,__median_absolute_deviation___4_,__median_absolute_deviation___5_,__percent_close_to_median___0_,__percent_close_to_median___1_,__percent_close_to_median___2_,__percent_close_to_median___3_,__percent_close_to_median___4_,__percent_close_to_median___5_,__percent_difference_flux_percentile___0_,__percent_difference_flux_percentile___1_,__percent_difference_flux_percentile___2_,__percent_difference_flux_percentile___3_,__percent_difference_flux_percentile___4_,__percent_difference_flux_percentile___5_,__percent_amplitude___0_,__percent_amplitude___1_,__percent_amplitude___2_,__percent_amplitude___3_,__percent_amplitude___4_,__percent_amplitude___5_,__qso_log_chi2_qsonu___0_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___2_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___4_,__qso_log_chi2_qsonu___5_,__qso_log_chi2nuNULL_chi2nu___0_,__qso_log_chi2nuNULL_chi2nu___1_,__qso_log_chi2nuNULL_chi2nu___2_,__qso_log_chi2nuNULL_chi2nu___3_,__qso_log_chi2nuNULL_chi2nu___4_,__qso_log_chi2nuNULL_chi2nu___5_,__skew___0_,__skew___1_,__skew___2_,__skew___3_,__skew___4_,__skew___5_,__std___0_,__std___1_,__std___2_,__std___3_,__std___4_,__std___5_,__stetson_j___0_,__stetson_j___1_,__stetson_j___2_,__stetson_j___3_,__stetson_j___4_,__stetson_j___5_,__stetson_k___0_,__stetson_k___1_,__stetson_k___2_,__stetson_k___3_,__stetson_k___4_,__stetson_k___5_,__freq1_amplitude1___0_,__freq1_amplitude1___1_,__freq1_amplitude1___2_,__freq1_amplitude1___3_,__freq1_amplitude1___4_,__freq1_amplitude1___5_,__freq1_amplitude2___0_,__freq1_amplitude2___1_,__freq1_amplitude2___2_,__freq1_amplitude2___3_,__freq1_amplitude2___4_,__freq1_amplitude2___5_,__freq1_amplitude3___0_,__freq1_amplitude3___1_,__freq1_amplitude3___2_,__freq1_amplitude3___3_,__freq1_amplitude3___4_,__freq1_amplitude3___5_,__freq1_amplitude4___0_,__freq1_amplitude4___1_,__freq1_amplitude4___2_,__freq1_amplitude4___3_,__freq1_amplitude4___4_,__freq1_amplitude4___5_,__freq1_freq___0_,__freq1_freq___1_,__freq1_freq___2_,__freq1_freq___3_,__freq1_freq___4_,__freq1_freq___5_,__freq1_rel_phase2___0_,__freq1_rel_phase2___1_,__freq1_rel_phase2___2_,__freq1_rel_phase2___3_,__freq1_rel_phase2___4_,__freq1_rel_phase2___5_,__freq1_rel_phase3___0_,__freq1_rel_phase3___1_,__freq1_rel_phase3___2_,__freq1_rel_phase3___3_,__freq1_rel_phase3___4_,__freq1_rel_phase3___5_,__freq1_rel_phase4___0_,__freq1_rel_phase4___1_,__freq1_rel_phase4___2_,__freq1_rel_phase4___3_,__freq1_rel_phase4___4_,__freq1_rel_phase4___5_,__freq2_amplitude

In [12]:
test_id = test_metadata_kaggle['object_id']

In [13]:
cesium_features = pd.read_csv('cesium_features.csv')
my_features = pd.read_csv('my_features.csv')

In [14]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [15]:
cesium_features['fold_sum'].value_counts()

2    121
1    105
3     50
0      6
Name: fold_sum, dtype: int64

In [16]:
columns_0 = list(cesium_features[cesium_features['fold_sum'] == 0]['column_name'])

In [18]:
columns_0.append('__stetson_k___2_')
columns_0.append('__freq3_amplitude1___1_')
columns_0.append('__median_absolute_deviation___2_')
columns_0.append('__percent_close_to_median___2_')

In [19]:
print(columns_0)

['__flux_percentile_ratio_mid50___5_', '__flux_percentile_ratio_mid65___2_', '__median_absolute_deviation___2_', '__qso_log_chi2_qsonu___0_', '__stetson_k___1_', '__freq1_signif___2_', '__stetson_k___2_', '__freq3_amplitude1___1_', '__median_absolute_deviation___2_', '__percent_close_to_median___2_']


In [20]:
cesium_imps = ['object_id','target','flux_max','flux_mean','flux_median','flux_skew','flux_err_min',
               'flux_err_mean','detected_mean','flux_by_flux_ratio_sq_skew','flux_w_mean','flux_diff1',
              'flux_diff2','flux_diff3','0__fft_coefficient__coeff_0__attr_"abs"',
               '0__fft_coefficient__coeff_1__attr_"abs"','0__skewness',
               '1__fft_coefficient__coeff_0__attr_"abs"','1__fft_coefficient__coeff_1__attr_"abs"',
               '1__kurtosis','1__skewness','2__fft_coefficient__coeff_0__attr_"abs"',
               '2__fft_coefficient__coeff_1__attr_"abs"','2__kurtosis','2__skewness',
               '3__fft_coefficient__coeff_0__attr_"abs"','3__fft_coefficient__coeff_1__attr_"abs"',
               '3__kurtosis','3__skewness','4__fft_coefficient__coeff_0__attr_"abs"',
               '4__fft_coefficient__coeff_1__attr_"abs"','4__kurtosis','4__skewness',
               '5__fft_coefficient__coeff_0__attr_"abs"','5__fft_coefficient__coeff_1__attr_"abs"',
               '5__kurtosis','5__skewness','flux__longest_strike_above_mean','flux__longest_strike_below_mean',
               'flux_by_flux_ratio_sq__longest_strike_above_mean','flux_by_flux_ratio_sq__longest_strike_below_mean',
               'mjd__mean_abs_change','mjd__mean_change','mjd_diff_det','hostgal_photoz','hostgal_photoz_err',
               'distmod','hostgal_photoz_certain','__amplitude___0_','__amplitude___1_',
               '__amplitude___2_','__amplitude___3_','__amplitude___4_','__amplitude___5_',
               '__flux_percentile_ratio_mid20___1_','__flux_percentile_ratio_mid20___2_',
               '__flux_percentile_ratio_mid35___1_','__flux_percentile_ratio_mid35___2_',
               '__max_slope___1_','__max_slope___2_','__max_slope___3_','__max_slope___5_',
               '__median_absolute_deviation___0_','__median_absolute_deviation___1_',
               '__median_absolute_deviation___2_','__median_absolute_deviation___3_',
               '__median_absolute_deviation___4_','__median_absolute_deviation___5_',
               '__percent_close_to_median___2_','__percent_close_to_median___3_',
               '__percent_close_to_median___4_','__percent_close_to_median___5_',
               '__percent_difference_flux_percentile___1_','__percent_difference_flux_percentile___2_',
               '__percent_difference_flux_percentile___3_','__percent_difference_flux_percentile___5_',
               '__percent_amplitude___1_','__percent_amplitude___2_','__qso_log_chi2_qsonu___0_',
               '__qso_log_chi2_qsonu___1_','__qso_log_chi2_qsonu___3_','__qso_log_chi2_qsonu___4_',
               '__qso_log_chi2_qsonu___5_','__skew___0_','__skew___1_','__skew___2_',
               '__skew___3_','__skew___4_','__skew___5_',
               '__std___0_','__std___1_','__std___2_','__std___3_',
               '__std___4_','__std___5_','__stetson_j___0_','__stetson_j___1_','__stetson_j___2_',
               '__stetson_j___3_','__stetson_j___4_','__stetson_j___5_','__stetson_k___0_',
               '__stetson_k___2_','__stetson_k___3_','__stetson_k___4_','__stetson_k___5_',
               '__freq1_amplitude1___1_','__freq1_amplitude1___5_','__freq1_amplitude3___3_',
               '__freq1_amplitude3___4_','__freq1_amplitude4___5_','__freq2_amplitude1___5_',
               '__freq2_amplitude2___5_','__freq3_amplitude1___0_','__freq_varrat___0_',
               '__freq_varrat___1_','__freq_varrat___2_','__freq_varrat___3_','__freq_varrat___4_',
               '__freq_varrat___5_'
              ]

In [21]:
cesium_imps2 = [x for x in cesium_imps if x not in columns_0 + ['object_id','target']]

In [22]:
cesium_imps2 = [x for x in cesium_imps2 if x in train_metadata.columns]

In [23]:
print(len(cesium_imps2))

68


In [24]:
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + columns_0 ],on = 'object_id',how = 'left')

In [25]:
%%time
final_dict = {}

loss_list = []
#temp = train_metadata_kaggle.copy()
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 500, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.1, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

0 0.662155061168403
1 0.6191581770694822
2 0.6793342588613245
3 0.6112001870680623
4 0.6139900864584982
MULTI WEIGHTED LOG LOSS : 0.63737 
CPU times: user 8min 15s, sys: 830 ms, total: 8min 16s
Wall time: 1min 6s


In [28]:
%%time
final_dict = {}
for column_ in cesium_imps2:
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id'] + columns_0 + [column_] ],on = 'object_id',how = 'left')
    

    loss_list = []
    #temp = train_metadata_kaggle.copy()
    #temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    # Taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    #print('Unique classes : ', classes)

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 500, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10, 
    'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    'reg_alpha': 0.1, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list

__amplitude___0_ MULTI WEIGHTED LOG LOSS : 0.63561 
__amplitude___1_ MULTI WEIGHTED LOG LOSS : 0.63490 
__amplitude___2_ MULTI WEIGHTED LOG LOSS : 0.63352 
__amplitude___3_ MULTI WEIGHTED LOG LOSS : 0.63475 
__amplitude___4_ MULTI WEIGHTED LOG LOSS : 0.63502 
__amplitude___5_ MULTI WEIGHTED LOG LOSS : 0.63566 
__flux_percentile_ratio_mid20___1_ MULTI WEIGHTED LOG LOSS : 0.63634 
__flux_percentile_ratio_mid20___2_ MULTI WEIGHTED LOG LOSS : 0.63560 
__flux_percentile_ratio_mid35___1_ MULTI WEIGHTED LOG LOSS : 0.63602 
__flux_percentile_ratio_mid35___2_ MULTI WEIGHTED LOG LOSS : 0.63603 
__max_slope___1_ MULTI WEIGHTED LOG LOSS : 0.63657 
__max_slope___2_ MULTI WEIGHTED LOG LOSS : 0.63425 
__max_slope___3_ MULTI WEIGHTED LOG LOSS : 0.63431 
__max_slope___5_ MULTI WEIGHTED LOG LOSS : 0.63524 
__median_absolute_deviation___0_ MULTI WEIGHTED LOG LOSS : 0.63572 
__median_absolute_deviation___1_ MULTI WEIGHTED LOG LOSS : 0.63686 
__median_absolute_deviation___3_ MULTI WEIGHTED LOG LOSS : 0.636

In [37]:
final_dict2 = pd.Series(final_dict)

In [38]:
final_dict2 = pd.DataFrame(final_dict2)

In [39]:
final_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])
final_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])
final_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])
final_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])
final_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])

In [40]:
del final_dict2[0]

In [41]:
final_dict2 = final_dict2.reset_index(drop=False)

In [42]:
final_dict2 = final_dict2.rename(columns={'index':'column_name'})

In [43]:
final_dict2['fold1_1'] = (final_dict2['fold1'] > 0.66215) * 1
final_dict2['fold2_1'] = (final_dict2['fold2'] > 0.61915) * 1
final_dict2['fold3_1'] = (final_dict2['fold3'] > 0.67933) * 1
final_dict2['fold4_1'] = (final_dict2['fold4'] > 0.61120) * 1
final_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61399) * 1
final_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']

In [44]:
final_dict2.head()

,column_name,fold1,fold2,fold3,fold4,fold5,fold1_1,fold2_1,fold3_1,fold4_1,fold5_1,fold_sum
0,__amplitude___0_,0.656316,0.621495,0.682503,0.611011,0.605606,0,1,1,0,0,2
1,__amplitude___1_,0.655486,0.622676,0.678000,0.609506,0.607610,0,1,0,0,0,1
2,__amplitude___2_,0.654242,0.621004,0.677131,0.608331,0.605653,0,1,0,0,0,1
3,__amplitude___3_,0.658166,0.623345,0.678774,0.607894,0.604255,0,1,0,0,0,1
4,__amplitude___4_,0.659924,0.620617,0.676092,0.608964,0.608068,0,1,0,0,0,1


In [45]:
final_dict2.to_csv('cesium_features_2.csv',index=False)

In [48]:
final_dict2[final_dict2['fold_sum'] == 0]

,column_name,fold1,fold2,fold3,fold4,fold5,fold1_1,fold2_1,fold3_1,fold4_1,fold5_1,fold_sum
66,__freq_varrat___4_,0.648047,0.61772,0.677394,0.605851,0.612149,0,0,0,0,0,0


# GOOD FEATURES

In [83]:
good_features = ['__freq_varrat___5_',
                 '__freq_varrat___4_',
                 '__qso_log_chi2_qsonu___3_',
                '__qso_log_chi2_qsonu___1_',
                 '__qso_log_chi2_qsonu___5_',
                '__std___4_',
                 '__freq_varrat___3_',
                 '__amplitude___2_'
                ]

In [84]:
#good_features = ['__freq_varrat___4_','__freq_varrat___5_']

In [85]:
%%time
final_dict = {}

loss_list = []
#temp = train_metadata_kaggle.copy()
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')

temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + columns_0 + good_features  ],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

print(temp.shape)

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 500, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.1, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 81)
0 0.6462657114002985
1 0.607965791018667
2 0.6748395712796345
3 0.601918405841696
4 0.6131804779219846
MULTI WEIGHTED LOG LOSS : 0.62891 
CPU times: user 7min 38s, sys: 824 ms, total: 7min 39s
Wall time: 1min


In [ ]:
#temp = temp[cesium_imps2]